# Classification and Regression, from linear and logistic regression to neural networks

## Regression:
Bring back regression code from project 1 (reference own work in bibliography) and compare to results from FFNN
Data can be:
- simple 1d
- then probably test with Franke before reading image data from cancer data set


## Classification:
Develop a logistic regression code and compare to FFNN.
Proposed data set:
- Wisconsin Breast Cancer Data (images representing various tumor features)
- Andre data (men pass på at kan diskutere resultatene mot literature)

## Part c) Testing different activation functions for the hidden layers
- Sigmoid
- RELU
- Leaky RELU
Discuss the results
May also study the way you initialise the  weights and biases

## Part d) Classification analysis using neural networks

## Part e) Write your logistic regression code
To compare the FFNN code with logistic regression

## Part f) Critical evaluation of the various algorithms
Compare the results from the different algorithms, which works best for the regression case, and which one for classification? List pros and cons








## Part a) Write own Stochastic Gradient Descent Code: First step
Replace matrix inversion from OLS and Ridge with GD and SGD
Data can be either
- Franke
- Terrain 
- Recommended: something simple like for instance: f(x) = ao + a1x + a2x^2
look at exercises from week 41 for help

## Analyse the GD and SGD code. This should include:
(Lecture notes week 39 and 40 contain examples it is ok to use)
### 1: plain gradient descent with fixed learning rate (tune it) using the analytical expression for the gradient
### 2: Add momentum to the plain GD code and compare convergence with a fixed learning rate (may need to tune it). Keep using the analytical expression for the gradient
### 3: Repeat these steps for stochastic gradient descent with mini batches and a given number of epochs. Use a tunable learning rate (lectures week 39 and 40). Discuss the results as functions of the various parameters (size of batches, nr of epochs etc). Use the analytical gradient
### 4: Implement the adagrad method in order to tune the learning rate. Do this with and without momentum for plain gradient descent and SGD
### 5: Add RMSprop and Adam to your library of methods for tuning the learning rate
### 6: Replace thereafter your analytical gradient with either Autograd or JAX

In [ ]:
from enum import Enum
from random import random, seed
import numpy as np

def MSE(expected_values, predicted_values):
    n = np.size(predicted_values)  # Number of data points
    return np.sum((expected_values - predicted_values)**2)/n

class LinearRegressionModel(Enum):
    OrdinaryLeastSquares = 0,
    Ridge = 1

class ScalingAlgorithm(Enum):
    GradientDescent = 0,
    MomentumGD = 1, #TODO: This is probably not relevant here anymore
    StochasticGD = 2,
    Adagrad = 3,
    RMSprop = 4,
    Adam = 5

class OLS_Hessian:
    def __init__(self, n, X):
        self.H = (2.0/n)* X.T @ X
        self.EigValues, self.EigVectors = np.linalg.eig(self.H)

class Ridge_Hessian:
    def __init__(self, n, lmbda, XT_X):
        self.H = (2.0/n)* XT_X+2*lmbda* np.eye(XT_X.shape[0])
        self.EigValues, self.EigVectors = np.linalg.eig(self.H)

class LinearRegression:
    def __init__(self, x, y):
        # Design matrix including the intercept
        # No scaling of data and all data used for training 
        n = x.shape[0]
        self.design_matrix = np.c_[np.ones((n, 1)), x, x*x]

        self.x = x
        self.y = y

        self.ols_hessian = None
        self.ridge_hessian = None
    
    def _grad(self, eta, gradient):
        return eta*gradient

    def _momentum_grad(self, eta, gradient, momentum, change):
        return eta*gradient + momentum*change

    def _adagrad(self, gradients, eta, G_iter):
        delta  = 1e-8 # AdaGrad parameter to avoid possible division by zero

        G_iter += gradients*gradients
        update = gradients*eta/(delta+np.sqrt(G_iter))

        return [update, G_iter]
    
    def gradient_descent_OLS(self, n, n_iterations, eta, scaling_algorithm = ScalingAlgorithm.GradientDescent, rho = None, momentum = None):
        #Don't wanna calculate this every time when running multiple iterations
        if not self.ols_hessian:
            self.ols_hessian = OLS_Hessian(n, self.design_matrix)
        
        betas, scores = list(), list()
        beta = np.random.randn(3,1)
        #eta = eta/np.max(self.ols_hessian.EigValues)
        change, G_iter = 0.0, np.zeros_like(beta)
        delta  = 1e-8

        for iter in range(n_iterations):
            gradient = (2.0/n)*self.design_matrix.T @ (self.design_matrix @ beta-self.y)
            match scaling_algorithm:
                case ScalingAlgorithm.GradientDescent:
                    beta -= self._grad(eta, gradient)

                case ScalingAlgorithm.MomentumGD:
                    new_change = self._momentum_grad(eta, gradient, momentum, change)
                    change = new_change
                    beta -= new_change

                case ScalingAlgorithm.Adagrad:
                    update, new_G_iter = self._adagrad(gradient, eta, G_iter)
                    G_iter = new_G_iter
                    if momentum:
                        update = momentum*change + update
                        change = update
                    beta -= update

                case ScalingAlgorithm.RMSprop:
                    if rho is None:
                        raise ValueError("Decay rate cannot be null when scaling with RMSprop")
                    
                    G_iter = rho*G_iter + (1-rho) * gradient**2
                    update = gradient*eta/(delta+np.sqrt(G_iter))
                    if momentum:
                        change = momentum*change - update
                        beta += change
                    else:
                        beta -= update
                
                case _:
                    raise NotImplementedError()
                
            #Store results
            betas.append(beta)
            scores.append(MSE(self.y, self.design_matrix @ beta))

        return [betas, scores]
    
    #Function contents taken from lecture notes
    def gradient_descent_ridge(self, n, n_iterations, eta, lmbda, momentum = None): 
        #Don't wanna calculate this every time when running multiple iterations
        if not self.ridge_hessian:
            XT_X = self.design_matrix.T @ self.design_matrix
            self.ridge_hessian = Ridge_Hessian(n, lmbda, XT_X)
        
        #Id = n*lmbda* np.eye((XT_X).shape[0])

        betas, scores = list(), list()
        beta = np.random.randn(3,1)
        #eta = eta/np.max(self.ridge_hessian.EigValues)
        change = 0.0
        for iter in range(n_iterations):
            gradient = 2.0/n*self.design_matrix.T @ (self.design_matrix @ beta-self.y)+2*lmbda*beta
            #Gradient descent with momentum:
            if momentum:
                new_change = eta*gradient + momentum*change
                change = new_change
                beta -= new_change

                #Store results
                betas.append(beta)
                scores.append(MSE(self.y, self.design_matrix @ beta))
            #Gradient descent without momentum:
            else:
                beta -= eta*gradient

                #Store results
                betas.append(beta)
                scores.append(MSE(self.y, self.design_matrix @ beta))

        return [betas, scores]
    
    def _learning_schedule(self, t):
        t0, t1 = 5, 50
        return t0/(t+t1)
    
    def _SGD(self, beta, lin_reg_model, epoch, M, m, scaling_algorithm = ScalingAlgorithm.StochasticGD, lmbda = None, rho = None, beta_values=None, momentum = None):
        G_iter = np.zeros_like(beta)
        change = np.zeros_like(beta) if momentum else None
        delta = 1e-8 #Constant for AdaGrad
        first_moment = np.zeros_like(beta)  # Initialize for Adam first moment
        second_moment = np.zeros_like(beta)  # Initialize for Adam second moment

        for iter in range(m):
            random_index = M*np.random.randint(m)
            xi = self.design_matrix[random_index:random_index+M]
            yi = self.y[random_index:random_index+M]

            y_tilde = xi @ beta
            gradients = (2.0/M)* xi.T @ (y_tilde-yi)

            match lin_reg_model:
                case LinearRegressionModel.Ridge:
                    if not lmbda:
                        raise ValueError("Lambda cannot be null for Ridge regression")
                    gradients += 2*lmbda*beta
            
            t = epoch*m+iter
            eta = self._learning_schedule(t)
            
            match scaling_algorithm:
                case ScalingAlgorithm.StochasticGD:
                    if momentum:
                        change = momentum*change - eta*gradients
                        beta += change
                    else:
                        beta -= eta*gradients

                case ScalingAlgorithm.Adagrad:
                    G_iter += gradients * gradients
                    update = gradients*eta/(delta+np.sqrt(G_iter))
                    if momentum:
                        change = momentum*change - update
                        beta += change
                    else:
                        beta -= update

                case ScalingAlgorithm.RMSprop:
                    if rho is None:
                        raise ValueError("Decay rate cannot be null when scaling with RMSprop")
                    G_iter = rho*G_iter + (1-rho) * gradients**2
                    update = gradients*eta/(delta+np.sqrt(G_iter))
                    if momentum:
                        change = momentum*change - update
                        beta += change
                    else:
                        beta -= update
                
                case ScalingAlgorithm.Adam:
                    if beta_values is None:
                        raise ValueError("Beta values must be provided to run with ADAM")
                    
                    #From ChatGPT:
                    t += 1  # Increment time step
                    first_moment = beta_values[0] * first_moment + (1 - beta_values[0]) * gradients  # Update first moment
                    second_moment = beta_values[1] * second_moment + (1 - beta_values[1]) * gradients**2  # Update second moment

                    # Bias correction
                    first_term = first_moment / (1 - beta_values[0]**t)  # Corrected first moment
                    second_term = second_moment / (1 - beta_values[1]**t)  # Corrected second moment

                    update = eta * first_term / (np.sqrt(second_term) + delta)  # Update parameters

                    beta -= update
                    
                case _:
                    raise NotImplementedError()

        return beta

    
    def SGD_OLS(self, n_datapoints, batch_size, n_epochs, scaling_algorithm = ScalingAlgorithm.StochasticGD, rho = None, beta_values = None, momentum = None):
        m = int(n_datapoints/batch_size)
        beta = np.random.randn(3,1)

        betas, scores = list(), list()
        for epoch in range(n_epochs):
            beta = self._SGD(beta, LinearRegressionModel.OrdinaryLeastSquares, epoch, batch_size, m, scaling_algorithm, rho=rho, beta_values=beta_values, momentum=momentum)
            
            #Store results
            betas.append(beta)
            scores.append(MSE(self.y, self.design_matrix @ beta))
        
        return [betas, scores]
    
    def SGD_Ridge(self, n_datapoints, batch_size, n_epochs, lmbda, scaling_algorithm = ScalingAlgorithm.StochasticGD):
        m = int(n_datapoints/batch_size)
        beta = np.random.randn(3,1)

        betas, scores = list(), list()
        for epoch in range(n_epochs):
            beta = self._SGD(beta, LinearRegressionModel.Ridge, epoch, batch_size, m, scaling_algorithm, lmbda)

            #Store results
            betas.append(beta)
            scores.append(MSE(self.y, self.design_matrix @ beta))

        return [betas, scores]    

In [ ]:
def Get_best_mse_and_eta(mse_data, learning_rates):
    min_mse = np.min(mse_data)
    best_index = np.argmin(mse_data)
    best_learning_rate = learning_rates[best_index]

    return [min_mse, best_learning_rate]

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

#Set-up taken from lecture notes
n = 100
x = 2*np.random.rand(n,1)
y = 4+3*x+5*x*x + np.random.randn(n, 1)

N_iterations = 30

linear_regression = LinearRegression(x, y)

In [ ]:
#Gradient descent
learning_rates = np.array([0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1])
mse_data = np.zeros((len(learning_rates), 2))  # Rows for learning rates, columns for OLS and Ridge
mse_data_test = list()
for i, eta in enumerate(learning_rates):
    ols_betas, ols_mses = linear_regression.gradient_descent_OLS(n, N_iterations, eta)
    mse_data[i, 0] = min(ols_mses)
    ridge_betas, ridge_mses = linear_regression.gradient_descent_ridge(n, N_iterations, eta, lmbda=0.01)
    mse_data[i, 1] = min(ridge_mses)

    # print(f"OLS - eta: {eta}, mse: {ols_mse}")
    # print(f"Ridge - eta: {eta}, mse: {ridge_mse}")


# Find the minimum MSE and corresponding index
ols_min_mse, ols_best_eta = Get_best_mse_and_eta(mse_data[:, 0], learning_rates)
ridge_min_mse, ridge_best_eta = Get_best_mse_and_eta(mse_data[:, 1], learning_rates)

# Print results
print(f"Best learning rate for OLS: {ols_best_eta} with MSE: {ols_min_mse}")
print(f"Best learning rate for Ridge: {ridge_best_eta} with MSE: {ridge_min_mse}")

# Get values for the best learning rate: (I'm lazy and run it again rather than rewrite the loop above to store it)
ols_betas, ols_mses = linear_regression.gradient_descent_OLS(n, N_iterations, ols_best_eta)
ridge_betas, ridge_mses = linear_regression.gradient_descent_ridge(n, N_iterations, ridge_best_eta, lmbda=0.01)


# Plot MSE over iterations
plt.plot(range(N_iterations), ols_mses, label="MSE (OLS) over iterations")
plt.plot(range(N_iterations), ridge_mses, label="MSE (Ridge) over iterations")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.title("Gradient Descent Optimization of MSE")
plt.legend()
plt.show()


# Create a DataFrame for heatmap plotting
mse_df = pd.DataFrame(mse_data, columns=['OLS MSE', 'Ridge MSE'], index=learning_rates)

# Plot the heatmap
plt.figure(figsize=(8, 6))
#sns.heatmap(mse_df, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'MSE'})
sns.heatmap(mse_df, annot=True, fmt=".3f", cmap="coolwarm", cbar=False)
plt.title("OLS and Ridge Regression Across Learning Rates")
plt.xlabel("Regression Type")
plt.ylabel("Learning Rates")
plt.xticks(ticks=np.arange(0.5, 2), labels=mse_df.columns, rotation=0)
plt.yticks(ticks=np.arange(len(learning_rates)), labels=mse_df.index, rotation=0)
plt.show()




In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

#Gradient descent with momentum
learning_rates = np.array([0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1])
mse_data = np.zeros((len(learning_rates), 2))  # Rows for learning rates, columns for OLS and Ridge
for i, eta in enumerate(learning_rates):
    #OLS
    ols_betas, ols_mses = linear_regression.gradient_descent_OLS(n, N_iterations, eta, ScalingAlgorithm.MomentumGD, momentum=momentum)
    mse_data[i, 0] = min(ols_mses)
    ridge_betas, ridge_mses = linear_regression.gradient_descent_ridge(n, N_iterations, eta, 0.01, momentum=momentum)
    mse_data[i, 1] = min(ridge_mses)

    #print(f"OLS - eta: {eta}, mse: {ols_mse}    Ridge - eta: {eta}, mse: {ridge_mse}")

# Find the minimum MSE and corresponding index
ols_min_mse, ols_best_eta = Get_best_mse_and_eta(mse_data[:, 0], learning_rates)
ridge_min_mse, ridge_best_eta = Get_best_mse_and_eta(mse_data[:, 1], learning_rates)

# Print results
print(f"Best learning rate for OLS: {ols_best_eta} with MSE: {ols_min_mse}")
print(f"Best learning rate for Ridge: {ridge_best_eta} with MSE: {ridge_min_mse}")

# Get values for the best learning rate: (I am lazy and run it again rather than rewrite the loop above to store it)
ols_betas, ols_mses = linear_regression.gradient_descent_OLS(n, N_iterations, ols_best_eta, ScalingAlgorithm.MomentumGD, momentum=momentum)
ridge_betas, ridge_mses = linear_regression.gradient_descent_ridge(n, N_iterations, ridge_best_eta, 0.01, momentum=momentum)


# Plot MSE over iterations
plt.plot(range(N_iterations), ols_mses, label="MSE (OLS) over iterations")
plt.plot(range(N_iterations), ridge_mses, label="MSE (Ridge) over iterations")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.title("Gradient Descent Optimization of MSE")
plt.legend()
plt.show()

# Create a DataFrame for heatmap plotting
mse_df = pd.DataFrame(mse_data, columns=['OLS MSE', 'Ridge MSE'], index=learning_rates)

# Plot the heatmap
plt.figure(figsize=(8, 6))
#sns.heatmap(mse_df, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'MSE'})
sns.heatmap(mse_df, annot=True, fmt=".3f", cmap="coolwarm", cbar=False)
plt.title("OLS and Ridge Regression Across Learning Rates w/ Memory")
plt.xlabel("Regression Type")
plt.ylabel("Learning Rates")
plt.xticks(ticks=np.arange(0.5, 2), labels=mse_df.columns, rotation=0)
plt.yticks(ticks=np.arange(len(learning_rates)), labels=mse_df.index, rotation=0)
plt.show()




In [ ]:
#SGD with batches and epocs

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Test for different nr epochs
epochs = np.array([1, 10, 50, 100, 500, 1000])
#.. and batch size
batch_sizes = np.array([1, 5, 10, 20, 50])
mse_data_ols = np.zeros((len(batch_sizes), len(epochs)))  # Arrays for epochs and batch sizes
mse_data_ridge = np.zeros((len(batch_sizes), len(epochs)))  # Arrays for epochs and batch sizes
for j, M in enumerate(batch_sizes):
    for i, n_epochs in enumerate(epochs):
        beta_ols, scores_ols = linear_regression.SGD_OLS(n, M, n_epochs)
        mse_data_ols[j, i] = min(scores_ols)

        beta_ridge, scores_ridge = linear_regression.SGD_Ridge(n, M, n_epochs, lmbda=0.01)
        mse_data_ridge[j, i] = min(scores_ridge)

# Find the minimum MSE and corresponding indices for OLS
ols_min_mse_index = np.unravel_index(np.argmin(mse_data_ols, axis=None), mse_data_ols.shape)
ols_min_mse = mse_data_ols[ols_min_mse_index]
ols_best_mini_batch_size = batch_sizes[ols_min_mse_index[0]]
ols_best_n_epochs = epochs[ols_min_mse_index[1]]

# Find the minimum MSE and corresponding indices for Ridge
ridge_min_mse_index = np.unravel_index(np.argmin(mse_data_ridge, axis=None), mse_data_ridge.shape)
ridge_min_mse = mse_data_ridge[ridge_min_mse_index]
ridge_best_mini_batch_size = batch_sizes[ridge_min_mse_index[0]]
ridge_best_n_epochs = epochs[ridge_min_mse_index[1]]

# Print results
print(f"Best settings for OLS: Mini-batch size = {ols_best_mini_batch_size}, Epochs = {ols_best_n_epochs} with MSE: {ols_min_mse}")
print(f"Best settings for Ridge: Mini-batch size = {ridge_best_mini_batch_size}, Epochs = {ridge_best_n_epochs} with MSE: {ridge_min_mse}")

# Create a DataFrame for heatmap plotting
mse_df_ols = pd.DataFrame(mse_data_ols, index=batch_sizes, columns=epochs)
mse_df_ridge = pd.DataFrame(mse_data_ridge, index=batch_sizes, columns=epochs)


# Plot the heatmap
plt.figure(figsize=(14, 6))

# OLS MSE Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(mse_df_ols, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'MSE'})
plt.title("OLS MSE Across Epochs and Mini-batch Sizes")
plt.xlabel("Nr of Epochs")
plt.ylabel("Mini-batch Size")

# Ridge MSE Heatmap
plt.subplot(1, 2, 2)
sns.heatmap(mse_df_ridge, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'MSE'})
plt.title("Ridge MSE Across Epochs and Mini-batch Sizes")
plt.xlabel("Nr of Epochs")
plt.ylabel("Mini-batch Size")

plt.tight_layout()
plt.show()


In [ ]:
#Adagrad
n = 100
n_iterations = 1000
eta = 0.1
momentum = 0.3
batch_size = 5
epochs = 100

#Regular adagrad
ada_betas, ada_mses = linear_regression.gradient_descent_OLS(n, n_iterations, eta, ScalingAlgorithm.Adagrad)
print(f"Best GD AdaGrad: {min(ada_mses)}")

#Adagrad with momentum
ada_betas, ada_mses = linear_regression.gradient_descent_OLS(n, n_iterations, eta, ScalingAlgorithm.Adagrad, momentum)
print(f"Best GD AdaGrad with momentum: {min(ada_mses)}")

#SGD with AdaGrad
ada_betas, ada_mses = linear_regression.SGD_OLS(n, batch_size, epochs, ScalingAlgorithm.Adagrad)
print(f"Best SGD AdaGrad: {min(ada_mses)}")

#SGD with AdaGrad and momentum
ada_betas, ada_mses = linear_regression.SGD_OLS(n, batch_size, epochs, ScalingAlgorithm.Adagrad, momentum=momentum)
print(f"Best SGD AdaGrad with momentum: {min(ada_mses)}")

#For comparison
#GD:
gd_betas, gd_mses = linear_regression.gradient_descent_OLS(n, n_iterations, eta, ScalingAlgorithm.GradientDescent)
print(f"Best GD: {min(gd_mses)}")

#GD with momentum:
gd_betas, gd_mses = linear_regression.gradient_descent_OLS(n, n_iterations, eta, ScalingAlgorithm.GradientDescent, momentum=momentum)
print(f"Best GD with momentum: {min(gd_mses)}")

#SGD:
sgd_betas, sgd_mses = linear_regression.SGD_OLS(n, batch_size, epochs)
print(f"Best SGD: {min(sgd_mses)}")

#SGD with momentum:
sgd_betas, sgd_mses = linear_regression.SGD_OLS(n, batch_size, epochs, momentum=momentum)
print(f"Best SGD with momentum: {min(sgd_mses)}")

In [ ]:
#RMSprop
betas, rms_prop_mses = linear_regression.gradient_descent_OLS(n=100, n_iterations=1000, eta=0.1, scaling_algorithm=ScalingAlgorithm.RMSprop, rho=0.9)
print(f"Best GD RMSprop: {min(rms_prop_mses)}")

#RMSprop with momentum
betas, rms_prop_mses = linear_regression.gradient_descent_OLS(n=100, n_iterations=1000, eta=0.1, scaling_algorithm=ScalingAlgorithm.RMSprop, rho=0.9, momentum=0.9)
print(f"Best GD RMSprop with momentum: {min(rms_prop_mses)}")

#SGD with RMSprop
betas, rms_prop_mses = linear_regression.SGD_OLS(n_datapoints=100, batch_size=5, n_epochs=100, scaling_algorithm=ScalingAlgorithm.RMSprop, rho=0.9)
print(f"Best SGD RMSprop: {min(rms_prop_mses)}")

#SGD with RMSprop with momentum
betas, rms_prop_mses = linear_regression.SGD_OLS(n_datapoints=100, batch_size=5, n_epochs=100, scaling_algorithm=ScalingAlgorithm.RMSprop, rho=0.9, momentum=0.9)
print(f"Best SGD RMSprop with momentum: {min(rms_prop_mses)}")

#SGD with ADAM
betas, rms_prop_mses = linear_regression.SGD_OLS(n_datapoints=100, batch_size=5, n_epochs=100, scaling_algorithm=ScalingAlgorithm.Adam, beta_values=[0.9, 0.999])
print(f"Best SGD ADAM: {min(rms_prop_mses)}")


## Part b) Writing your own neural network code (Central part of the project)
Implement back propagation algorithm discussed in lectures week 41 and 42
### Regression problem first: Can use simple 2nd order poly from part a, Franke or terrain. Discuss choice of cost fcn
- Decide on cost fcn
- With Sigmoid as activation fcn for hidden layers, and a flexible nr of hidden layers, write an FFNN
- Initialise wheights using a normal distribution
- How would you initialise the biases?
- Which activation would you choose for the final output layer?
- Train your network

If running on Franke:
- compare your results from OLS and Ridge from project 1 (if using franke or terrain)

But also:
- Compare to similar code in scikit
Comment on results and compare linear regression to this FFNN
Make an analysis of the regularisation parameters and the learning rates employed to find the optimal MSE and R2 scores.


In [61]:
##FFNN
from typing import List
from enum import Enum
from random import random, seed
import numpy as np

class ActivationFunction(Enum):
    Sigmoid = 0,
    RELU = 1,
    LeakyRELU = 2

class CostFunction(Enum):
    MSE = 0,
    R2 = 1

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z)) 

def sigmoid_derivative(a):
    return a * (1 - a)

class NeuralNetwork:
    def __init__(self, 
                 #output_layer, 
                 cost_function, 
                 #Output layer:
                 outputlayer_activation = None, 
                 n_output_neurons = 1, 
                 n_outputs = 1, 
                 #Hidden layer:
                 hidden_activation = ActivationFunction.Sigmoid,
                 hidden_features = 1,
                 hidden_neurons = 1,
                 hidden_layers = None):
        #self.input_layer = input_layer
        self.hidden_layers = hidden_layers
        self.cost_function = cost_function
        output_layer = OutputLayer(outputlayer_activation, n_outputs)

        #User can self-define a list of hidden layers, otherwise the default is one layer with given values
        if hidden_layers == None:
            self.hidden_layers = list([HiddenLayer(hidden_activation, hidden_features, hidden_neurons)])
        
        n_neurons_last_hiddenlayer = self.hidden_layers[-1]._get_hidden_neurons()
        output_layer._set_weights(n_neurons_last_hiddenlayer)
        self.output_layer = output_layer
            
    
    #So as to be flexible, we can expand the model after creation - will probably need a method like GetHiddenLayers
    def add_hidden_layer(self, position, layer):
        self.hidden_layers.append(layer) #TODO: At a specific position
            
    def evaluate_model(self, predicted_value, target_value):
        match self.cost_function:
            case CostFunction.MSE:
                return np.mean((predicted_value - target_value) ** 2)
            case _:
                raise NotImplementedError(f"Cost function {self.cost_function} not implemented.")

    def FeedForward(self, x):
        activations = []
        a = x  # Initial input is x
        for i, layer in enumerate(self.hidden_layers):
            z = np.matmul(a, layer.get_weights()) + layer.get_biases()
            a = layer.activation(z)
            activations.append(a)  # Collect activation from each layer for backprop
        # Output layer computation (no activation here for regression)
        z_out = np.matmul(a, self.output_layer.get_weights()) + self.output_layer.get_biases()
        activations.append(z_out)
        return activations
    
    def BackPropagation(self, activations, y, x):
        # Initializing gradients
        output_weights_gradient = []
        output_bias_gradient = []
        
        hidden_weights_gradient = []
        hidden_bias_gradient = []
        
        a_output = activations[-1]
        delta_output = (a_output - y) * self.output_layer.derivative(a_output)

        #print(0.5 * ((a_output - y) ** 2))  # MSE for linear output
        
        # Backpropagate through hidden layers
        delta = np.matmul(delta_output, self.output_layer.get_weights().T) * self.hidden_layers[-1].derivative(activations[-2])
        output_weights_gradient = np.matmul(activations[-2].T, delta_output)
        output_bias_gradient = np.sum(delta_output, axis=0, keepdims=True)

        for i in reversed(range(len(self.hidden_layers))):
            # Insert at start of list because working backwards
            hidden_weights_gradient.insert(0, np.matmul(activations[i-1].T, delta) if i > 0 else np.matmul(x.T, delta))
            hidden_bias_gradient.insert(0, np.sum(delta, axis=0, keepdims=True))

            # Prepare delta for the next layer back
            if i > 0:
                delta = np.matmul(delta, self.hidden_layers[i].get_weights().T) * self.hidden_layers[i-1].derivative(activations[i-1])

        
        return output_weights_gradient, output_bias_gradient, hidden_weights_gradient, hidden_bias_gradient
    
    # Update weights and biases using gradients
    def update_parameters(self, gradients, learning_rate):
        output_weights_gradient, output_bias_gradient, hidden_weights_gradient, hidden_bias_gradient = gradients
        
        # Update output layer weights and biases
        self.output_layer.w -= learning_rate * output_weights_gradient
        self.output_layer.b -= learning_rate * output_bias_gradient.squeeze()
        
        # Update hidden layer weights and biases
        for i, layer in enumerate(self.hidden_layers):
            layer.w -= learning_rate * hidden_weights_gradient[i]
            layer.b -= learning_rate * hidden_bias_gradient[i].squeeze()

    
    # Train the model
    def train(self, x, y, epochs, learning_rate):
        final_loss = 0
        #Becomes relevant if adding a stop condition:
        final_epoch = 0
        for epoch in range(epochs):
            final_epoch = epoch + 1
            # Feedforward pass
            activations = self.FeedForward(x)
            # Compute loss for monitoring (optional)
            loss = self.evaluate_model(activations[-1], y)
            final_loss = loss
            #print(f"Epoch {epoch+1}, Loss: {loss}")
            # Backpropagation
            gradients = self.BackPropagation(activations, y, x)
            # Update weights and biases
            self.update_parameters(gradients, learning_rate)

        return [final_epoch, final_loss]
    
class Layer:
    def __init__(self, activation_function, n_features, n_hidden_neurons):
        # weights and bias in the hidden layer
        self.w = np.random.randn(n_features, n_hidden_neurons)
        #self.b = np.zeros(n_hidden_neurons) #+ 0.01 
        self.b = np.zeros((1, n_hidden_neurons))
        #TODO: For different activation functions, a small initial value may be beneficial, especially for relu to avoid vanishing neurons
        self.activation_fnc = activation_function
        self.n_neurons = n_hidden_neurons

    def get_weights(self):
        return self.w
    
    def get_biases(self):
        return self.b
    
    def activation(self, z):
        match self.activation_fnc:
            case ActivationFunction.Sigmoid:
                return sigmoid(z)
            case None:
                return z # Typical for regression type problems
            case _:
                raise NotImplementedError(f"Activation function not implemented for: {self.activation_fnc}")

    def derivative(self, a):
        match self.activation_fnc:
            case ActivationFunction.Sigmoid:
                return sigmoid_derivative(a)
            case None:
                return np.ones_like(a) #Linear #TODO: Is this right?
            case _:
                raise NotImplementedError(f"Derivative function not implemented for {self.activation_fnc}")

class HiddenLayer(Layer):
    pass
    def _get_hidden_neurons(self):
        return self.n_neurons

class OutputLayer(Layer):
    def __init__(self, activation_function, n_outputs):
        # weights and bias in the output layer
        #self.b = np.zeros(n_outputs) #+ 0.01 
        self.n_outputs = n_outputs
        self.b = np.zeros((1, n_outputs))
        self.activation_fnc = activation_function

    def _set_weights(self, n_hidden_neurons):
        self.w = np.random.randn(n_hidden_neurons, self.n_outputs)

#TODO: Input layer if/when needed
    


In [62]:
import numpy as np
# We use the Sigmoid function as activation function

# ensure the same random numbers appear every time
np.random.seed(0)
# Input variable
x = np.array([4.0],dtype=np.float64)
# Target values
y = 2*x+1.0 
y += np.random.normal(0, 100, y.shape)

# Defining the neural network, only scalars here
n_inputs = x.shape
n_features = 1
n_hidden_neurons = 1
n_outputs = 1

n_iterations = 50
#TODO: Some stopping condition

#Set up and train simple model
model = NeuralNetwork(CostFunction.MSE)
epochs, loss = model.train(x, y, n_iterations, learning_rate=0.1)


print(f"Epoch {epochs}, Loss: {loss}")


Epoch 50, Loss: 1.1280593056418667e-05


In [63]:
#Compare to scikit learn

import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

# 1. Generate synthetic data for regression
# Ensure the same random numbers appear every time
np.random.seed(0)

X, y = make_regression(n_samples=100, n_features=1, noise=10)

# 2. Initialize and fit the Scikit-learn MLPRegressor
scikit_model = MLPRegressor(hidden_layer_sizes=(1,),  # You can change the size and number of hidden layers
                             learning_rate_init=0.001,
                             max_iter=50,
                             random_state=0)

scikit_model.fit(X, y)

# 3. Predictions and evaluation using Scikit-learn
y_pred_sklearn = scikit_model.predict(X)
mse_sklearn = mean_squared_error(y, y_pred_sklearn)
print(f"Mean Squared Error with Scikit-learn: {mse_sklearn}")

# 4. Initialize and train custom neural network
hidden_layers = list([HiddenLayer(ActivationFunction.Sigmoid, n_features=1, n_hidden_neurons=5)])
custom_model = NeuralNetwork(CostFunction.MSE, hidden_layers=hidden_layers)

epoch, loss = custom_model.train(X, y.reshape(-1, 1), epochs=50, learning_rate=0.001)  # Reshape y to match expected input
print(f"Mean Squared Error with custom neural net: {loss}")

# iterations = 300
# for iter in iterations:
    

Mean Squared Error with Scikit-learn: 1953.0277026047493
Mean Squared Error with custom neural net: 236.97401363469467


/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
